In [2]:
import asyncio
import json
import logging
import os
from datetime import datetime, timezone
from logging import INFO
import google.generativeai as genai

from dotenv import load_dotenv

from graphiti_core import Graphiti
from graphiti_core.nodes import EpisodeType
from graphiti_core.search.search_config_recipes import NODE_HYBRID_SEARCH_RRF
from neo4j import AsyncGraphDatabase


logging.basicConfig(
    level=INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
)
logger = logging.getLogger(__name__)

load_dotenv()

# Set-up Neo4j and Graphiti 
neo4j_uri = os.environ.get('NEO4J_URI', 'bolt://localhost:7687')
neo4j_user = os.environ.get('NEO4J_USER', 'neo4j')
neo4j_password = os.environ.get('NEO4J_PASSWORD', 'password')

# Set-up Gemini 
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)
gemini_model = genai.GenerativeModel("gemini-2.5-flash")




In [7]:
async def main():
    # Main function implementation will go here
    pass

await main()


In [8]:
# Initialize Graphiti with Neo4j connection
graphiti = Graphiti(neo4j_uri, neo4j_user, neo4j_password)

try:
    # Initialize the graph database with graphiti's indices. This only needs to be done once.
    await graphiti.build_indices_and_constraints()

    # Additional code will go here

finally:
    # Close the connection
    await graphiti.close()
    print('\nConnection closed')



Connection closed


In [9]:
# Episodes list containing both text and JSON episodes
episodes = [
    {
        'content': 'Kamala Harris is the Attorney General of California. She was previously '
        'the district attorney for San Francisco.',
        'type': EpisodeType.text,
        'description': 'podcast transcript',
    },
    {
        'content': 'As AG, Harris was in office from January 3, 2011 – January 3, 2017',
        'type': EpisodeType.text,
        'description': 'podcast transcript',
    },
    {
        'content': {
            'name': 'Gavin Newsom',
            'position': 'Governor',
            'state': 'California',
            'previous_role': 'Lieutenant Governor',
            'previous_location': 'San Francisco',
        },
        'type': EpisodeType.json,
        'description': 'podcast metadata',
    },
    {
        'content': {
            'name': 'Gavin Newsom',
            'position': 'Governor',
            'term_start': 'January 7, 2019',
            'term_end': 'Present',
        },
        'type': EpisodeType.json,
        'description': 'podcast metadata',
    },
]

# Add episodes to the graph
for i, episode in enumerate(episodes):
    await graphiti.add_episode(
        name=f'Freakonomics Radio {i}',
        episode_body=episode['content']
        if isinstance(episode['content'], str)
        else json.dumps(episode['content']),
        source=episode['type'],
        source_description=episode['description'],
        reference_time=datetime.now(timezone.utc),
    )
    print(f'Added episode: Freakonomics Radio {i} ({episode["type"].value})')


2025-07-03 23:45:17 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-03 23:45:19 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: Unknown function 'vector.similarity.cosine' (line 4, column 17 (offset: 79))
"        WITH n, vector.similarity.cosine(n.name_embedding, $search_vector) AS score"
                 ^}

In [10]:
print("\nSearching for: 'Who was the California Attorney General?'")
results = await graphiti.search('Who was the California Attorney General?')

# Print search results
print('\nSearch Results:')
for result in results:
    print(f'UUID: {result.uuid}')
    print(f'Fact: {result.fact}')
    if hasattr(result, 'valid_at') and result.valid_at:
        print(f'Valid from: {result.valid_at}')
    if hasattr(result, 'invalid_at') and result.invalid_at:
        print(f'Valid until: {result.invalid_at}')
    print('---')



Searching for: 'Who was the California Attorney General?'


2025-07-03 23:45:58 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Exception ignored in: <coroutine object main at 0x130ddcd50>
Traceback (most recent call last):
  File "<string>", line 1, in <lambda>
KeyError: '__import__'
Exception ignored in: <coroutine object main at 0x130ddcd50>
Traceback (most recent call last):
  File "<string>", line 1, in <lambda>
KeyError: '__import__'


CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: Unknown function 'vector.similarity.cosine' (line 4, column 26 (offset: 115))
"        WITH DISTINCT r, vector.similarity.cosine(r.fact_embedding, $search_vector) AS score"
                          ^}

In [ ]:
if results and len(results) > 0:

    center_node_uuid = results[0].source_node_uuid

    print('\nReranking search results based on graph distance:')
    print(f'Using center node UUID: {center_node_uuid}')

    reranked_results = await graphiti.search(
        'Who was the California Attorney General?', center_node_uuid=center_node_uuid
    )

    print('\nReranked Search Results:')
    for result in reranked_results:
        print(f'UUID: {result.uuid}')
        print(f'Fact: {result.fact}')
        if hasattr(result, 'valid_at') and result.valid_at:
            print(f'Valid from: {result.valid_at}')
        if hasattr(result, 'invalid_at') and result.invalid_at:
            print(f'Valid until: {result.invalid_at}')
        print('---')
else:
    print('No results found in the initial search to use as center node.')


In [ ]:
# Example: Perform a node search using _search method with standard recipes
print(
    '\nPerforming node search using _search method with standard recipe NODE_HYBRID_SEARCH_RRF:'
)

# Use a predefined search configuration recipe and modify its limit
node_search_config = NODE_HYBRID_SEARCH_RRF.model_copy(deep=True)
node_search_config.limit = 5  # Limit to 5 results

# Execute the node search
node_search_results = await graphiti._search(
    query='California Governor',
    config=node_search_config,
)

# Print node search results
print('\nNode Search Results:')
for node in node_search_results.nodes:
    print(f'Node UUID: {node.uuid}')
    print(f'Node Name: {node.name}')
    node_summary = node.summary[:100] + '...' if len(node.summary) > 100 else node.summary
    print(f'Content Summary: {node_summary}')
    print(f"Node Labels: {', '.join(node.labels)}")
    print(f'Created At: {node.created_at}')
    if hasattr(node, 'attributes') and node.attributes:
        print('Attributes:')
        for key, value in node.attributes.items():
            print(f'  {key}: {value}')
    print('---')


In [3]:
from graphiti_core.nodes import EpisodeType

print([e for e in EpisodeType])


[<EpisodeType.message: 'message'>, <EpisodeType.json: 'json'>, <EpisodeType.text: 'text'>]


In [5]:
import inspect
print(inspect.signature(Graphiti.add_episode))

(self, name: str, episode_body: str, source_description: str, reference_time: datetime.datetime, source: graphiti_core.nodes.EpisodeType = <EpisodeType.message: 'message'>, group_id: str = '', uuid: str | None = None, update_communities: bool = False, entity_types: dict[str, pydantic.main.BaseModel] | None = None, excluded_entity_types: list[str] | None = None, previous_episode_uuids: list[str] | None = None, edge_types: dict[str, pydantic.main.BaseModel] | None = None, edge_type_map: dict[tuple[str, str], list[str]] | None = None) -> graphiti_core.graphiti.AddEpisodeResults


In [ ]:
1/0

In [ ]:
class ChatMemory:
    def __init__(self, uri, user, password):
        self.driver = AsyncGraphDatabase.driver(uri, auth=(user, password))
        self.graphiti = Graphiti(uri=uri, user=user, password=password)

    async def save_message_as_episode(self, user_id: str, text: str, sender: str):
        timestamp = datetime.now(timezone.utc)

        await self.graphiti.add_episode(
            name=f"{user_id}_{sender}_{timestamp.isoformat()}",
            episode_body=text,
            source=EpisodeType.text,
            source_description=f"chat message from {sender}",
            reference_time=timestamp
        )


async def run_conversation():
    memory = ChatMemory(neo4j_uri, neo4j_user, neo4j_password)

    user_id = "user-123"
    user_input = "I need help with my order"

    await memory.save_message_as_episode(user_id, user_input, sender="user")

    response = gemini_model.generate_content(user_input)
    reply = response.text.strip()

    await memory.save_message_as_episode(user_id, reply, sender="agent")

    print("Gemini Reply:", reply)
    return response


response = await run_conversation()
print(response)